# Setup

## Import packages

In [ ]:
import os
import joblib
from datetime import datetime

import numpy as np
import pandas as pd
from evidently.dashboard import Dashboard
from evidently.model_profile import Profile
from evidently.profile_sections import (DataDriftProfileSection,
                                        NumTargetDriftProfileSection,
                                        RegressionPerformanceProfileSection)
from evidently.tabs import (DataDriftTab, NumTargetDriftTab,
                            RegressionPerformanceTab)


## Install and Enable Evidently Jupyter Notebook Extension

In [ ]:
!jupyter nbextension install --sys-prefix --symlink --overwrite --py evidently
!jupyter nbextension enable evidently --py --sys-prefix

# Read the Data

Since this is a sample, we are reading data from a csv file. In a production scenario, we could read the reference data from the production bucket

In [ ]:
reference_df = pd.read_csv('train.csv')
current_df = pd.read_csv('test.csv')

In [ ]:
reference_df.info()

In [ ]:
# Create a column mapping dictionary and assign values.
# This will also be used by Evidently later to perform the appropriate statistical tests.
column_mapping={}

column_mapping['target'] = 'Churn'


column_mapping['numerical_features'] = ['MonthlyCharges', 'TotalCharges']

column_mapping['categorical_features'] = ['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'Tenure',
                                          'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                                          'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
                                          'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

In [ ]:
# Set target and feature variables
target = column_mapping['target']
features = column_mapping['numerical_features'] + column_mapping['categorical_features']

# Load the Model

In [ ]:
model = joblib.load('model.joblib')

# Perform Predictions

## On Train Data

In [ ]:
reference_df['prediction'] = model.predict(reference_df.drop('Churn', axis=1))

In [ ]:
column_mapping['prediction'] = 'prediction'

## On Test Data

In [ ]:
current_df['prediction'] = model.predict(current_df.drop('Churn', axis=1))

# Evidently Dashboards

## Regression Performance

In [ ]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab])
regression_perfomance_dashboard.calculate(reference_df, current_df, column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.save('performance.html')

In [ ]:
regression_perfomance_dashboard.show()

## Data Drift

In [ ]:
drift_dashboard = Dashboard(tabs=[DataDriftTab])
drift_dashboard.calculate(reference_df, current_df)

In [ ]:
drift_dashboard.show()

## Target Drift

In [ ]:
drift_dashboard.save('drift.html')

In [ ]:
concept_drift_dashboard = Dashboard(tabs=[NumTargetDriftTab])
concept_drift_dashboard.calculate(reference_df, current_df)

## Bringing it all together

In [ ]:
evidently_dashboard = Dashboard(tabs=[NumTargetDriftTab, DataDriftTab, RegressionPerformanceTab])
evidently_dashboard.calculate(reference_df, current_df, column_mapping)

In [ ]:
evidently_dashboard.save('dashboard.html')